In [1]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import random
import pickle
import heapq
import sys
import os
import gc

np.random.seed(42)
random.seed(42)

In [2]:
root_path = 'E:\\Tasks\\08. Task Allocation with TCC'
code_path = os.path.join(root_path, 'Codes')
results_path = os.path.join(root_path, 'Resuts')

sys.path.insert(0,code_path)
from Configuration import node_removal_id, node_addition_id, number_of_iterations, time_step, th_size, save_object, load_object, W1, W2, W3
from CostFunction import CostFunction
from Task import Task
from Node import Node

In [3]:
def make_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

simulated_annealing_path = os.path.join(results_path, "simulated_annealing")
single_simulated_annealing_path = os.path.join(results_path, "single_simulated_annealing")
random_path = os.path.join(results_path, "random")
greedy_path = os.path.join(results_path, "greedy")

make_folder(results_path)
make_folder(simulated_annealing_path)
make_folder(single_simulated_annealing_path)
make_folder(random_path)
make_folder(greedy_path)

In [4]:
def tasks_genration_model(number_of_tasks):
    return [Task() for i in range(number_of_tasks)]

def nodes_generation_model(number_of_nodes):
    return [Node() for i in range(number_of_nodes)]

In [5]:
def generate_initial_solution(nodes_set, tasks_set):
    solution = np.zeros((len(nodes_set), len(tasks_set), len(tasks_set[-1].sub_tasks)))
    for k in range(len(tasks_set[-1].sub_tasks)):
        for j, task in enumerate(tasks_set):
            sub_task = task.sub_tasks[k]
            for i, node in enumerate(nodes_set):
                if (not node.is_terminated) and (not sub_task.is_allocated):
                    if node.computational_capacities>=node.used_computational+sub_task.computation_load:
                        if node.start_time==0:
                            node.start_time=((sub_task.computation_load/node.processing_speed)*sub_task.priorty)
                        else:
                            node.start_time+=(sub_task.computation_load/node.processing_speed)
                        node.used_computational+=sub_task.computation_load
                        sub_task.is_allocated = True
                        solution[i][j][k]=1
                        node.tasks.append((j, k))
    return solution, nodes_set, tasks_set

In [6]:
def active_nodes(nodes_set):
  return [node for node in nodes_set if len(node.tasks)>0]

In [7]:
def evaluate_function(nodes_set, tasks_set, cost_function, single=False):
    the_active_nodes = active_nodes(nodes_set)
    execution_time = cost_function.totale_execution_time(the_active_nodes, tasks_set)
    execution_energy = cost_function.totale_execution_energy(the_active_nodes, tasks_set)
    energy_distribution = cost_function.calculate_energy_distribution(the_active_nodes)
    execution_time_distribution = cost_function.calculate_execution_time_distribution(the_active_nodes)
    if single:
        return ((W1*execution_time_distribution) + (W2*energy_distribution) + (W3*len(the_active_nodes)),), execution_time_distribution, energy_distribution, len(the_active_nodes)
    return execution_time_distribution, energy_distribution, len(the_active_nodes)

In [8]:
def task_allocation_ratio(tasks_set):
  number_of_sub_tasks, number_of_alloctaed_sub_task = len(tasks_set)*3, 0
  for i, task in enumerate(tasks_set):
    for sub_task in task.sub_tasks:
      if sub_task.is_allocated:number_of_alloctaed_sub_task+=1
  return number_of_alloctaed_sub_task/number_of_sub_tasks

In [9]:
def allocated_tasks(tasks_set):
    is_allocated, allocated_tasks, allocated_sub_tasks = [], [], []
    for task in tasks_set:
        for sub in task.sub_tasks:
            allocated_sub_tasks.append(sub)
            is_allocated.append(sub.is_allocated)
        if np.all(is_allocated):
            allocated_tasks.append(task)
    return allocated_tasks, allocated_sub_tasks

In [10]:
def identify_best_node_for_remove(nodes_set):
    nodes_speed, indeces = [], []
    for index, node in enumerate(nodes_set):
        if (not node.is_terminated) and (node.tasks!=[]):
            nodes_speed.append(node.processing_speed)
            indeces.append(index)
    if nodes_speed==[]:return 0
    return indeces[nodes_speed.index(min(nodes_speed))]

In [11]:
def find_best_node_for_reallocate(nodes_set, task, node_for_removal_index):
    computation_capacities, indeces = [], []
    for index, node in enumerate(nodes_set):
        if index==node_for_removal_index:continue
        c = node.computational_capacities-(node.used_computational+task.computation_load)
        if c>0 and (not node.is_terminated):
            computation_capacities.append(c)
            indeces.append(index)
    if indeces==[]:return -1
    return indeces[computation_capacities.index(min(computation_capacities))]

In [12]:
def node_removal(nodes_set, tasks_set, current_solution, cost_function):
    node_for_remove_index = identify_best_node_for_remove(nodes_set)
    try:
        allocated_tasks = nodes_set[node_for_remove_index].tasks
    except:
        print(node_for_remove_index)
        print(len(nodes_set))
        allocated_tasks = nodes_set[node_for_remove_index].tasks
    if allocated_tasks==[]:
      for i, task in enumerate(allocated_tasks):
          new_node_index = find_best_node_for_reallocate(nodes_set, tasks_set[task[0]].sub_tasks[task[1]], node_for_remove_index)
          if new_node_index!=-1:
              nodes_set[new_node_index].start_time+=(tasks_set[task[0]].sub_tasks[task[1]].computation_load/nodes_set[new_node_index].processing_speed)
              nodes_set[node_for_remove_index].start_time-=(tasks_set[task[0]].sub_tasks[task[1]].computation_load/nodes_set[node_for_remove_index].processing_speed)
              nodes_set[new_node_index].tasks.append(task)
              current_solution[new_node_index][task[0]][task[1]]=1
              current_solution[node_for_remove_index][task[0]][task[1]]=0
              cost_function.reduce_execution_time(nodes_set[node_for_remove_index], node_for_remove_index, tasks_set[task[0]].sub_tasks[task[1]], task[0])
              del nodes_set[node_for_remove_index].tasks[i]
    if nodes_set[node_for_remove_index].tasks==[]:nodes_set[node_for_remove_index].is_terminated=False
    return nodes_set, tasks_set, current_solution

In [13]:
def allocate_tasks_not_allocated(nodes_set, tasks_set, new_node, current_solution):
    sub_solution, allocate = np.zeros((1, len(tasks_set), len(tasks_set[-1].sub_tasks))), False
    for k in range(len(tasks_set[-1].sub_tasks)):
        for j, task in enumerate(tasks_set):
            sub_task = task.sub_tasks[k]
            if not sub_task.is_allocated:
                if new_node.computational_capacities>=new_node.used_computational+sub_task.computation_load:
                    new_node.used_computational+=sub_task.computation_load
                    sub_task.is_allocated = True
                    sub_solution[0][j][k]=1
                    new_node.tasks.append((j, k))
                    allocate=True
                    if new_node.start_time==0:
                        new_node.start_time=((sub_task.computation_load/new_node.processing_speed)*sub_task.priorty)
                    else:
                        new_node.start_time+=(sub_task.computation_load/new_node.processing_speed)
                new_node.tasks = list(set(new_node.tasks))
    if allocate:
        nodes_set.append(new_node)
        current_solution = np.concatenate((current_solution, sub_solution), axis=0)
    return nodes_set, tasks_set, current_solution, allocate

In [14]:
def find_best_task_for_reallocate(nodes_set, tasks_set, new_node, current_solution, cost_function):
    sub_solution = np.zeros((1, len(tasks_set), len(tasks_set[-1].sub_tasks)))
    number_of_task = []
    for index, node in enumerate(nodes_set):
        number_of_task.append(len(node.tasks))
    max_five_indices = heapq.nlargest(5, range(len(number_of_task)), key=lambda i: number_of_task[i])
    for ind in max_five_indices:
        if number_of_task[ind]>=2 and new_node.computational_capacities-new_node.used_computational>=0:
            task_index = nodes_set[ind].tasks[0][0]
            sub_tasl_index = nodes_set[ind].tasks[0][1]
            sub_task = tasks_set[task_index].sub_tasks[sub_tasl_index]
            if new_node.computational_capacities>=new_node.used_computational+sub_task.computation_load:
                new_node.used_computational+=sub_task.computation_load
                sub_solution[0][task_index][sub_tasl_index] = 1
                current_solution[ind][task_index][sub_tasl_index] = 0
                new_node.tasks.append((task_index, sub_tasl_index))
                sub_task.is_allocated=True
                if new_node.start_time==0:
                    new_node.start_time=((sub_task.computation_load/new_node.processing_speed)*sub_task.priorty)
                else:
                    new_node.start_time+=(sub_task.computation_load/new_node.processing_speed)
                nodes_set[ind].start_time-=(sub_task.computation_load/nodes_set[ind].processing_speed)
                del nodes_set[ind].tasks[0]
                cost_function.reduce_execution_time(nodes_set[ind], ind, sub_task, task_index)
            new_node.tasks = list(set(new_node.tasks))
    if new_node.tasks!=[]:
        nodes_set.append(new_node)
        current_solution = np.concatenate((current_solution, sub_solution), axis=0)
    return nodes_set, tasks_set, current_solution

In [15]:
def dominates(solution1, solution2):
    return all(solution1[i] <= solution2[i] for i in range(len(solution1))) and any(solution1[i] < solution2[i] for i in range(len(solution1)))

In [16]:
def is_non_dominated(solution, population):
    for ind in population:
        if dominates(ind, solution):
            return False
    return True

In [17]:
def non_dominated_sort(population):
    fronts = []
    dominating_count = [0] * len(population)
    dominated_solutions = [[] for _ in range(len(population))]
    for i, p in enumerate(population):
        for j, q in enumerate(population):
            if dominates(p, q):
                dominating_count[i] += 1
            elif dominates(q, p):
                dominated_solutions[i].append(j)
    current_front = []
    for i, count in enumerate(dominating_count):
        if count == 0:
            current_front.append(i)
    fronts.append(current_front)
    while current_front:
        next_front = []
        for i in current_front:
            for j in dominated_solutions[i]:
                dominating_count[j] -= 1
                if dominating_count[j] == 0:
                    next_front.append(j)
        if next_front:
            fronts.append(next_front)
        current_front = next_front
    return fronts

In [18]:
def node_addtion(nodes_set, tasks_set, current_solution, cost_function):
    number_of_nodes = len(nodes_set)
    new_node = Node(number_of_nodes)
    cost_function.expand_matrixes()
    nodes_set, tasks_set, current_solution, allocate = allocate_tasks_not_allocated(nodes_set, tasks_set, new_node, current_solution)
    if not allocate:
        nodes_set, tasks_set, current_solution = find_best_task_for_reallocate(nodes_set, tasks_set, new_node, current_solution, cost_function)
    number_of_nodes = len(nodes_set)
    return nodes_set, tasks_set, current_solution

In [19]:
def generate_nieghbor_solution(nodes_set, tasks_set, current_solution, cost_function):
    operation_id = random.randint(0, 1)
    if operation_id == node_removal_id:
        nodes_set, tasks_set, current_solution = node_removal(nodes_set, tasks_set, current_solution, cost_function)
    elif operation_id == node_addition_id:
        nodes_set, tasks_set, current_solution = node_addtion(nodes_set, tasks_set, current_solution, cost_function)
    return nodes_set, tasks_set, current_solution

In [20]:
def applay_solution(nodes_set, tasks_set, solution, cost_function):
    cost_function.set_matrixes(len(nodes_set), len(tasks_set))
    for i, node in enumerate(solution):
        nodes_set[i].tasks = []
        nodes_set[i].used_computational = 0
        nodes_set[i].start_time = 0
        for j, task in enumerate(node):
            for k, sub_task in enumerate(task):
                if sub_task==1:
                    nodes_set[i].is_terminated = False
                    tasks_set[j].sub_tasks[k].is_allocated = True
                    nodes_set[i].tasks.append((j, k))
                    nodes_set[i].used_computational+=tasks_set[j].sub_tasks[k].computation_load
                    if nodes_set[i].start_time==0:
                        nodes_set[i].start_time=((tasks_set[j].sub_tasks[k].computation_load/nodes_set[i].processing_speed)*tasks_set[j].sub_tasks[k].priorty)
                    else:
                        nodes_set[i].start_time+=(tasks_set[j].sub_tasks[k].computation_load/nodes_set[i].processing_speed)
    return nodes_set, tasks_set, solution

In [21]:
def crowding_distance(solutions):
    solutions = np.array(solutions)
    num_solutions = len(solutions)
    num_objectives = len(solutions[0])
    crowding_distances = np.zeros(num_solutions)
    sorted_indices = np.argsort(solutions, axis=0)
    for obj_index in range(num_objectives):
        sorted_solutions = solutions[sorted_indices[:, obj_index]]
        crowding_distances[sorted_indices[0, obj_index]] = np.inf
        crowding_distances[sorted_indices[num_solutions - 1, obj_index]] = np.inf
        min_obj_val = sorted_solutions[0, obj_index]
        max_obj_val = sorted_solutions[num_solutions - 1, obj_index]
        if max_obj_val == min_obj_val:
            continue  # Avoid division by zero
        for i in range(1, num_solutions - 1):
            crowding_distances[sorted_indices[i, obj_index]] += \
                (sorted_solutions[i + 1, obj_index] - sorted_solutions[i - 1, obj_index]) / (max_obj_val - min_obj_val)
    return [index for index, value in enumerate(crowding_distances) if value <= 0.2]

In [22]:
def remove_duplicates_in_order(lst):
    seen = set()
    result = []
    for i, item in enumerate(lst):
        if item not in seen:
            seen.add(item)
            result.append(i)
    return result

In [23]:
def delete_items_randomly(lst, num_items):
    if num_items >= len(lst):
        return []
    indices_to_delete = random.sample(range(len(lst)), num_items)
    indices_to_delete.sort(reverse=True)
    return indices_to_delete

In [24]:
def simulated_annealing(tasks_set, nodes_set, cost_function, number_of_iterations, th_size=100):
    pareto_front, objective_values, tasks_sets, nodes_sets = [], [], [], [] # tasks_allocation_ratio , []
    current_solution, nodes_set, tasks_set = generate_initial_solution(nodes_set.copy(), tasks_set.copy())
    objective_values.append(evaluate_function(nodes_set.copy(), tasks_set.copy(), cost_function))
    # tasks_allocation_ratio.append(task_allocation_ratio(tasks_set.copy()))
    pareto_front.append(current_solution.copy())
    nodes_sets.append(nodes_set.copy())
    tasks_sets.append(tasks_set.copy())
    for iteration in tqdm(range(number_of_iterations)):
      for index, _ in enumerate(pareto_front):
          none_duplicates = remove_duplicates_in_order(objective_values)
          objective_values = [objective_values[i] for i in none_duplicates]
          pareto_front = [pareto_front[i].copy() for i in none_duplicates]
          # tasks_allocation_ratio = [tasks_allocation_ratio[i] for i in none_duplicates]
          nodes_sets = [nodes_sets[i].copy() for i in none_duplicates]
          tasks_sets = [tasks_sets[i].copy() for i in none_duplicates]
          if index>=len(pareto_front):break
          current_solution, nodes_set, tasks_set = pareto_front[index].copy(), nodes_sets[index].copy(), tasks_sets[index].copy()
          nodes_set, tasks_set, _ = applay_solution(nodes_set, tasks_set, current_solution, cost_function)
          nodes_set, tasks_set, current_solution = generate_nieghbor_solution(nodes_set.copy(), tasks_set.copy(), current_solution.copy(),cost_function)
          objective_values.append(evaluate_function(nodes_set.copy(), tasks_set.copy(), cost_function))
          # tasks_allocation_ratio.append(task_allocation_ratio(tasks_set.copy()))
          pareto_front.append(current_solution.copy())
          nodes_sets.append(nodes_set.copy())
          tasks_sets.append(tasks_set.copy())
          non_dominated_indices = non_dominated_sort(objective_values)
          objective_values = [objective_values[i] for i in non_dominated_indices[0]]
          pareto_front = [pareto_front[i].copy() for i in non_dominated_indices[0]]
          # tasks_allocation_ratio = [tasks_allocation_ratio[i] for i in non_dominated_indices[0]]
          nodes_sets = [nodes_sets[i].copy() for i in non_dominated_indices[0]]
          tasks_sets = [tasks_sets[i].copy() for i in non_dominated_indices[0]]
          if len(pareto_front)>th_size:
            crowding_distance_indices = crowding_distance(objective_values)
            objective_values = [objective_values[i] for i in crowding_distance_indices]
            # tasks_allocation_ratio = [tasks_allocation_ratio[i] for i in crowding_distance_indices]
            pareto_front = [pareto_front[i].copy() for i in crowding_distance_indices]
            nodes_sets = [nodes_sets[i].copy() for i in crowding_distance_indices]
            tasks_sets = [tasks_sets[i].copy() for i in crowding_distance_indices]
    return pareto_front, objective_values, nodes_sets, tasks_sets

In [25]:
def single_simulated_annealing(tasks_set, nodes_set, cost_function, number_of_iterations, th_size=100):
    pareto_front, objective_values, tasks_sets, nodes_sets = [], [], [], []
    current_solution, nodes_set, tasks_set = generate_initial_solution(nodes_set.copy(), tasks_set.copy())
    objective_values.append(evaluate_function(nodes_set.copy(), tasks_set.copy(), cost_function, True))
    pareto_front.append(current_solution.copy())
    nodes_sets.append(nodes_set.copy())
    tasks_sets.append(tasks_set.copy())
    for iteration in tqdm(range(number_of_iterations)):
      for index, _ in enumerate(pareto_front):
          none_duplicates = remove_duplicates_in_order(objective_values)
          objective_values = [objective_values[i] for i in none_duplicates]
          pareto_front = [pareto_front[i].copy() for i in none_duplicates]
          nodes_sets = [nodes_sets[i].copy() for i in none_duplicates]
          tasks_sets = [tasks_sets[i].copy() for i in none_duplicates]
          if index>=len(pareto_front):break
          current_solution, nodes_set, tasks_set = pareto_front[index].copy(), nodes_sets[index].copy(), tasks_sets[index].copy()
          nodes_set, tasks_set, _ = applay_solution(nodes_set, tasks_set, current_solution, cost_function)
          nodes_set, tasks_set, current_solution = generate_nieghbor_solution(nodes_set.copy(), tasks_set.copy(), current_solution.copy(), cost_function)
          objective_values.append(evaluate_function(nodes_set.copy(), tasks_set.copy(), cost_function, True))
          pareto_front.append(current_solution.copy())
          nodes_sets.append(nodes_set.copy())
          tasks_sets.append(tasks_set.copy())
          if len(pareto_front)>th_size:
            crowding_distance_indices = delete_items_randomly(objective_values, len(pareto_front)-th_size)
            objective_values = [objective_values[i] for i in crowding_distance_indices]
            pareto_front = [pareto_front[i].copy() for i in crowding_distance_indices]
            nodes_sets = [nodes_sets[i].copy() for i in crowding_distance_indices]
            tasks_sets = [tasks_sets[i].copy() for i in crowding_distance_indices]
    return pareto_front, objective_values, nodes_sets, tasks_sets

In [26]:
def random_method(nodes_set, tasks_set, cost_function, number_of_iterations):
    shape = (len(nodes_set), len(tasks_set), (len(tasks_set[-1].sub_tasks)))
    pareto_front, objective_values, tasks_sets, nodes_sets = [], [], [], [] # tasks_allocation_ratio , []
    for i in tqdm(range(number_of_iterations)):
        operation_id = random.randint(0, 1)
        if operation_id == node_removal_id:
            # del nodes_set[random.randint(0, len(nodes_set)-1)]
            shape = (len(nodes_set), len(tasks_set), (len(tasks_set[-1].sub_tasks)))
        elif operation_id == node_addition_id:
            nodes_set.append(Node(len(nodes_set)))
            cost_function.set_matrixes(len(nodes_set), len(tasks_set))
            shape = (len(nodes_set), len(tasks_set), (len(tasks_set[-1].sub_tasks)))
            number_of_nodes = len(nodes_set)
        current_solution = np.random.randint(2, size=shape)
        nodes_set, tasks_set, current_solution = applay_solution(nodes_set, tasks_set, current_solution, cost_function)
        pareto_front.append(current_solution)
        objective_values.append(evaluate_function(nodes_set, tasks_set, cost_function))
        nodes_sets.append(nodes_set.copy())
        tasks_sets.append(tasks_set.copy())
        # tasks_allocation_ratio.append(task_allocation_ratio(tasks_set))
    return pareto_front, objective_values, nodes_sets, tasks_sets

In [27]:
def greedy_method(nodes_set, tasks_set, cost_function, number_of_iterations):
    pareto_front, objective_values, tasks_sets, nodes_sets = [], [], [], [] # tasks_allocation_ratio , []
    for i in tqdm(range(number_of_iterations)):
        operation_id = random.randint(0, 1)
        if operation_id == node_removal_id:
            # del nodes_set[random.randint(0, len(nodes_set)-1)]
            number_of_nodes = len(nodes_set)
        elif operation_id == node_addition_id:
            nodes_set.append(Node(len(nodes_set)))
            number_of_nodes = len(nodes_set)
            cost_function.set_matrixes(len(nodes_set), len(tasks_set))
        
        nodes_set = list(set(nodes_set))
        nodes_set = random.sample(nodes_set, len(nodes_set))
        
        current_solution, nodes_set, tasks_set = generate_initial_solution(nodes_set, tasks_set)
        pareto_front.append(current_solution)
        objective_values.append(evaluate_function(nodes_set, tasks_set, cost_function))
        nodes_sets.append(nodes_set.copy())
        tasks_sets.append(tasks_set.copy())
        # tasks_allocation_ratio.append(task_allocation_ratio(tasks_set))
    return pareto_front, objective_values, nodes_sets, tasks_sets

In [28]:
def get_best_objectives(objective_values):
    obj1 = [sol[0] for sol in objective_values]
    obj2 = [sol[1] for sol in objective_values]
    obj3 = [sol[2] for sol in objective_values]
    return obj1.index(min(obj1)), obj2.index(min(obj2)), obj3.index(min(obj3))

In [29]:
def index_of_tuple_with_greatest_min(tuples):
    max_count = -1
    index = -1
    for i, t in enumerate(tuples):
        min_val = min(t)
        count = t.count(min_val)
        if count > max_count:
            max_count = count
            index = i
        elif count == max_count:
            if sum(t) < sum(tuples[index]):
                index = i
    return index

In [30]:
def select_the_final_solution(nodes_sets, tasks_sets, objective_values):
    if len(objective_values[0])>3:
        objective_values = [(y[1], y[2], y[3]) for y in objective_values]
    while True:
        index = index_of_tuple_with_greatest_min(objective_values)
        tasks = tasks_sets[index]
        nodes = nodes_sets[index]
        bool_allocate = False
        for t in tasks:
            for tt in t.sub_tasks:
                if not tt.is_allocated:bool_allocate=True
        if not bool_allocate:break
        else:
            objective_values[index] = (10000000000, 10000000000, 10000000000)
    return nodes, tasks 

In [31]:
def reschedulezation(nodes_set, tasks_set):
    new_tasks_set, new_nodes_set = [], []
    for task_index, task in enumerate(tasks_set):
        temp_node, temp_sub_task = [], []
        for node in nodes_set:
            for t in node.tasks:
                if t[0]==task_index:
                   temp_node.append(node)
                   temp_sub_task.append(t[1])
        time_cost = 0
        computation_cost = []
        for node, sub_task in zip(temp_node, temp_sub_task):
            time_cost+=task.sub_tasks[sub_task].computation_load/node.processing_speed
            computation_cost.append(time_step * node.processing_speed)
        if time_cost>time_step:
            task.computation_load-=sum(computation_cost)
            for i, i_s_t in enumerate(temp_sub_task):
                task.sub_tasks[i_s_t].computation_load -= computation_cost[i]
            new_tasks_set.append(task)
    new_tasks_set = new_tasks_set + tasks_genration_model(random.randint(10, 30))
    for node in nodes_set:
        if not node.is_terminated:
            node.used_computational = 0
            node.start_time = 0
            node.tasks = []
            new_nodes_set.append(node)
    return new_nodes_set, new_tasks_set

In [32]:
def simulated_annealing_main(nodes_set, tasks_set, step):
    cost_function = CostFunction(len(nodes_set), len(tasks_set))
    pareto_front, objective_values, nodes_sets, tasks_sets = simulated_annealing(tasks_set, nodes_set, cost_function, number_of_iterations)
    print(objective_values)
    save_object(pareto_front, 'pareto_front_'+str(step), simulated_annealing_path)
    save_object(objective_values, 'objective_values_'+str(step), simulated_annealing_path)
    nodes_set, tasks_set = select_the_final_solution(nodes_sets, tasks_sets, objective_values)
    nodes_set, tasks_set = reschedulezation(nodes_set, tasks_set)
    gc.collect()
    return nodes_set, tasks_set

In [33]:
def random_main(nodes_set, tasks_set, step):
    cost_function = CostFunction(len(nodes_set), len(tasks_set))
    random_pareto_front, random_objective_values, nodes_sets, tasks_sets = random_method(nodes_set, tasks_set, cost_function, number_of_iterations)
    print(random_objective_values)
    save_object(random_pareto_front, 'pareto_front_'+str(step), random_path)
    save_object(random_objective_values, 'objective_values_'+str(step), random_path)
    nodes_set, tasks_set = select_the_final_solution(nodes_sets, tasks_sets, random_objective_values)
    nodes_set, tasks_set = reschedulezation(nodes_set, tasks_set)
    gc.collect()
    return nodes_set, tasks_set

In [34]:
def greedy_main(nodes_set, tasks_set, step):
    cost_function = CostFunction(len(nodes_set), len(tasks_set))
    greedy_pareto_front, greedy_objective_values, nodes_sets, tasks_sets = greedy_method(nodes_set, tasks_set, cost_function, number_of_iterations)
    print(greedy_objective_values)
    save_object(greedy_pareto_front, 'pareto_front_'+str(step), greedy_path)
    save_object(greedy_objective_values, 'objective_values_'+str(step), greedy_path)
    nodes_set, tasks_set = select_the_final_solution(nodes_sets, tasks_sets, greedy_objective_values)
    nodes_set, tasks_set = reschedulezation(nodes_set, tasks_set)
    gc.collect()
    return nodes_set, tasks_set

In [35]:
def single_simulated_annealing_main(nodes_set, tasks_set, step):
    cost_function = CostFunction(len(nodes_set), len(tasks_set))
    single_pareto_front, single_objective_values, nodes_sets, tasks_sets = single_simulated_annealing(tasks_set, nodes_set, cost_function, number_of_iterations)
    print(single_objective_values)
    save_object(single_pareto_front, 'pareto_front_'+str(step), single_simulated_annealing_path)
    save_object(single_objective_values, 'objective_values_'+str(step), single_simulated_annealing_path)
    nodes_set, tasks_set = select_the_final_solution(nodes_sets, tasks_sets, single_objective_values)
    nodes_set, tasks_set = reschedulezation(nodes_set, tasks_set)
    gc.collect()
    return nodes_set, tasks_set

In [36]:
numer_of_steps = 50
number_of_nodes = random.randint(5, 25)
nodes_set = nodes_generation_model(number_of_nodes)

number_of_tasks = random.randint(10, 30)
tasks_set = tasks_genration_model(number_of_tasks)
simulated_annealing_nodes, random_nodes, greedy_nodes, single_simulated_annealing_nodes = nodes_set.copy(), nodes_set.copy(), nodes_set.copy(), nodes_set.copy()
simulated_annealing_tasks, random_tasks, greedy_tasks, single_simulated_annealing_tasks = tasks_set.copy(), tasks_set.copy(), tasks_set.copy(), tasks_set.copy()


In [37]:
# for t in range(numer_of_steps):
   # print(25*"*", t, 25*"*")
    # simulated_annealing_nodes, simulated_annealing_tasks = simulated_annealing_main(simulated_annealing_nodes, simulated_annealing_tasks, t)

In [38]:
# for t in range(numer_of_steps):
    # print(25*"*", t, 25*"*")
    # random_nodes, random_tasks = random_main(random_nodes, random_tasks, t)

In [39]:
for t in range(numer_of_steps):
    print(25*"*", t, 25*"*")
    greedy_nodes, greedy_tasks = greedy_main(greedy_nodes, greedy_tasks, t)

************************* 0 *************************



[(17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.41671669925546, 5), (17.31470482360156, 406.416716


[(13.090532271504522, 400.77193800956604, 9), (13.090532271504522, 400.77193800956604, 9), (13.090532271504523, 400.77193800956604, 9), (13.090532271504522, 400.77193800956604, 9), (13.090532271504523, 400.77193800956604, 9), (13.090532271504523, 400.77193800956604, 9), (13.090532271504522, 400.77193800956604, 9), (13.090532271504523, 400.77193800956604, 9), (13.090532271504523, 400.77193800956604, 9), (13.090532271504522, 400.77193800956604, 9), (13.090532271504522, 400.77193800956604, 9), (13.090532271504523, 400.77193800956604, 9), (13.090532271504523, 400.77193800956604, 9), (13.090532271504523, 400.77193800956604, 9), (13.090532271504522, 400.77193800956604, 9), (13.090532271504522, 400.77193800956604, 9), (13.090532271504522, 400.77193800956604, 9), (13.090532271504522, 400.77193800956604, 9), (13.090532271504522, 400.77193800956604, 9), (13.090532271504522, 400.77193800956604, 9), (13.090532271504523, 400.77193800956604, 9), (13.090532271504523, 400.771938009566, 9), (13.090532


[(6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710509012697, 265.3445459604546, 8), (6.003710


[(3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.9085348827465642, 286.71977339816016, 4), (3.90853

...


[(11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.121850137430577, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.12185013743058, 364.2886123382498, 5), (11.1218


[(11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.77595355338985, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.87236185973848, 393.7759535533898, 7), (11.8723


[(9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.16884844504669, 202.51773324201906, 6), (9.168848


[(9.857009299340207, 336.7594894896045, 9), (9.85700929934021, 336.7594894896045, 9), (9.85700929934021, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.85700929934021, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.85700929934021, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.85700929934021, 336.7594894896045, 9), (9.857009299340207, 336.7594894896045, 9), (9.85700929934021, 336.7594894896045, 9), (9.857009299340207, 336.75948948960445, 9), (9.857009299340207, 336.7594894896045, 9), (9.857009299340207, 336.75948948960445, 9), (9.8570092993


[(17.564066970376363, 449.0364322189246, 9), (17.564066970376366, 449.0364322189245, 9), (17.564066970376366, 449.0364322189245, 9), (17.564066970376363, 449.0364322189246, 9), (17.564066970376366, 449.0364322189245, 9), (17.564066970376363, 449.0364322189245, 9), (17.564066970376366, 449.0364322189245, 9), (17.564066970376366, 449.0364322189245, 9), (17.564066970376363, 449.0364322189245, 9), (17.564066970376366, 449.0364322189245, 9), (17.564066970376366, 449.0364322189245, 9), (17.564066970376363, 449.0364322189245, 9), (17.564066970376366, 449.0364322189245, 9), (17.564066970376363, 449.03643221892446, 9), (17.564066970376363, 449.0364322189245, 9), (17.564066970376363, 449.0364322189245, 9), (17.564066970376363, 449.0364322189245, 9), (17.564066970376366, 449.0364322189245, 9), (17.564066970376363, 449.0364322189245, 9), (17.564066970376366, 449.0364322189245, 9), (17.564066970376363, 449.0364322189245, 9), (17.564066970376363, 449.0364322189245, 9), (17.564066970376363, 449.0364

In [40]:
# for t in range(numer_of_steps):
#     print(25*"*", t, 25*"*")
#     single_simulated_annealing_nodes, single_simulated_annealing_tasks= single_simulated_annealing_main(single_simulated_annealing_nodes, single_simulated_annealing_tasks, t)

************************* 0 *************************



[((53.1213795697401,), 3.1452112127504823, 136.5927206863832, 28), ((54.66814812229857,), 3.224896062208396, 140.33559767878683, 29), ((51.33066401814043,), 3.427767048263206, 129.00777967887157, 29), ((44.82106839724611,), 2.8443990336622096, 106.5591622904915, 30), ((52.58215816907248,), 3.2107348776809963, 133.39645901922728, 29), ((46.119248698530754,), 2.6706952618552924, 111.06013373324721, 30), ((46.75997690567546,), 2.7084658152338736, 113.15812387035099, 30), ((45.27118020181232,), 3.170896355162745, 107.73303765087833, 30), ((49.89781695571947,), 3.0793547797506045, 121.91336840598098, 31), ((51.057425978818294,), 3.3293386304402883, 126.86208129895402, 30), ((48.155676398986586,), 2.922767791042969, 116.26282020557899, 31), ((46.75997690567546,), 2.7084658152338736, 113.15812387035099, 30)]
************************* 1 *************************



[((52.075361727954565,), 4.382452423628476, 127.8687533362201, 31), ((52.602816423175184,), 3.835910932540851, 128.84014381137644, 32), ((46.847689917818656,), 3.2993354071823213, 108.85963098554653, 33), ((58.37362580230409,), 4.222698430973066, 147.68938757670722, 32), ((47.644567904016704,), 3.33511492813803, 111.48011141858431, 33), ((48.53475180566713,), 3.1248982587741825, 113.3242744267829, 34), ((51.32808140192139,), 3.6270231230714574, 123.46658154999983, 33), ((48.87752859587134,), 2.9143574207504335, 114.67740456548736, 34), ((48.87752859587134,), 2.9143574207504335, 114.67740456548736, 34)]
************************* 2 *************************



[((51.319439866243286,), 3.5182773375065186, 118.21318888330445, 37), ((52.65417477878938,), 3.8411378971250882, 121.0061113655062, 38), ((52.20559284865431,), 3.6459235382457744, 118.37271929060195, 39)]
************************* 3 *************************



[((50.256061819373656,), 2.920653121395635, 109.93288627651657, 41)]
************************* 4 *************************


...


[((46.44370755463583,), 3.4173731490555506, 98.06165203306388, 40), ((50.66389435495296,), 4.2961202981691, 109.9168608850074, 41), ((47.43296429629483,), 3.389354967329817, 100.05385935365292, 41), ((52.061651956496746,), 4.1329663233717095, 113.40587353161742, 42), ((49.02917550448403,), 3.6866786750621365, 105.07723967321796, 41), ((55.85359804737098,), 3.8005773704812253, 125.0447494540887, 43), ((50.38773851411584,), 3.6961334743879832, 109.5963282393315, 41), ((53.95457199530141,), 4.120201905389095, 119.72837141228229, 42), ((51.59063827010142,), 3.4407850661383548, 112.52800916753307, 42), ((52.56766000027885,), 3.7292130759692355, 112.82965359162694, 44), ((55.52907337184982,), 4.3432973602624845, 124.75361387923694, 42), ((53.426872972523455,), 3.8575094481558034, 118.23206712692237, 42), ((51.45058220790805,), 3.3663841202454052, 108.13555657278143, 45)]
************************* 46 *************************



[((50.23982385313498,), 4.355799531448575, 104.44361331233469, 44), ((55.28109166109471,), 4.72338053587613, 119.54692500110625, 45), ((55.33037845246315,), 4.437931684272656, 118.66332982393784, 46), ((60.63590643521964,), 5.51259838892519, 136.6070897284736, 45), ((55.99422000823168,), 4.53631687490616, 120.77774981919943, 46), ((54.809388631973405,), 4.457253233994782, 116.9073755392499, 46), ((58.88800400841609,), 4.973790939532106, 129.98622242185485, 46), ((51.55657807503586,), 4.084038197046377, 105.10455538640647, 47), ((54.03970496727588,), 4.4948308178745044, 112.97085240637844, 47), ((52.99479691784742,), 3.986434525757182, 108.6628885337342, 48)]
************************* 47 *************************



[((49.23747858389392,), 3.126539160435341, 106.3317227858777, 41), ((51.699186202235325,), 2.9520659836485765, 113.37855469046919, 42), ((52.66681094511958,), 4.245570813636858, 115.31046567009508, 42), ((48.50480055603983,), 2.940847779656965, 101.40848740714247, 43), ((48.82318829828364,), 2.8689408812540265, 102.54168677969146, 43), ((46.054133466444654,), 2.7104159210773395, 92.13669563373817, 44), ((46.940957252429726,), 2.6618136704199364, 95.14137717101248, 44), ((47.44082472424153,), 3.5135696306151725, 94.62251278352326, 45), ((51.43211607471527,), 4.128615240527824, 109.97843834185643, 43), ((47.91628202588793,), 3.1482146807289793, 97.90605873889746, 44), ((45.67205015294031,), 2.767862034503284, 89.47230514196441, 45), ((55.29002700001507,), 4.73455304000553, 123.56553696004472, 42), ((48.68676171680383,), 2.8323887230251876, 102.12348366632092, 43), ((46.839104693700925,), 2.761161687755524, 94.70252062458088, 44), ((45.77472717670263,), 2.7220282998844496, 89.86039562245


[((56.809803442222105,), 5.2102486780075115, 129.48909612939948, 41), ((51.58745939686639,), 4.290504464692123, 111.66769352486254, 42), ((59.638698995763264,), 5.315215457894249, 137.48044786131666, 42), ((48.651858241515825,), 3.8883003733676023, 100.95122709835181, 43), ((59.5059056281448,), 4.856454641489905, 137.49656411899278, 42), ((49.564794179582975,), 3.7830190233063936, 104.09962824197021, 43), ((48.72837917410804,), 3.6568419441146798, 100.10442196957881, 44), ((52.11613827474626,), 4.175961396029239, 112.21116618645833, 43), ((59.122744682803244,), 4.7280333461024995, 136.34778226324167, 42), ((48.94384378295827,), 3.826922536068491, 101.9858900737924, 43), ((52.21367399874066,), 4.555324864931501, 112.15692179753736, 43), ((50.466078779640306,), 4.030507420231053, 105.5230885119033, 44), ((54.16919687556583,), 4.39014802544697, 118.84050822643916, 43), ((49.77280996671137,), 3.809476186168648, 102.09989036953593, 45), ((51.381588706443296,), 3.957798344582366, 108.647497


[((48.81020302575487,), 4.099032359784677, 103.93497772606487, 41), ((48.81020302575487,), 4.099032359784677, 103.93497772606487, 41)]
